## Generate Example

In [ ]:
export PGHOST=db
export PGUSER=postgres
export PGDATABASE=postgres

In [ ]:
psql << EOM
    DROP TABLE IF EXISTS seq_data;
    SELECT *
    INTO seq_data
    FROM generate_series(1,10000) a, generate_series(1,100) b;

    DROP TABLE IF EXISTS rand_data;
    SELECT a, floor(random() * 99)::int b
    INTO rand_data
    FROM generate_series(1,1000000) a;
EOM

In [ ]:
psql << EOM
    DROP INDEX IF EXISTS idx_seq_data_a;
    DROP INDEX IF EXISTS idx_rand_data_a;
EOM

In [ ]:
psql << EOM
    CREATE INDEX idx_seq_data_a ON seq_data(a);
    CREATE INDEX idx_rand_data_a ON rand_data(a);
EOM

## Scans

### All

In [ ]:
psql -c "EXPLAIN SELECT * FROM seq_data;"

In [ ]:
psql -c "EXPLAIN SELECT * FROM rand_data;"

### With Filter

In [ ]:
# Added cpu_operator_cost
psql -c "EXPLAIN SELECT * FROM seq_data WHERE a = 200;"

In [ ]:
# Added cpu_operator_cost
psql -c "EXPLAIN SELECT * FROM rand_data WHERE a = 200;"

### With index and a filter

In [ ]:
psql -c "EXPLAIN SELECT * FROM seq_data WHERE a = 200;"

In [ ]:
psql -c "EXPLAIN SELECT * FROM rand_data WHERE a = 200;"

## Joins

### Types
- Run with different disable options

In [ ]:
# DISABLE_OTHER_JOINS="SET enable_hashjoin = off;SET enable_mergejoin = off;" # Nested
# DISABLE_OTHER_JOINS="SET enable_nestloop = off;SET enable_mergejoin = off;" # Hash
# DISABLE_OTHER_JOINS="SET enable_hashjoin = off;SET enable_nestloop = off;" # Merge
DISABLE_OTHER_JOINS="SET enable_hashjoin = on;SET enable_nestloop = on;SET enable_mergejoin = on;" # Enable all

Run two times:
1. Without Index
1. With Index

In [ ]:
# Get all data
psql -c "${DISABLE_OTHER_JOINS} EXPLAIN SELECT * FROM rand_data r JOIN seq_data s USING (a);"

In [ ]:
# Get only column a
psql -c "${DISABLE_OTHER_JOINS} EXPLAIN SELECT r.a, s.a FROM rand_data r JOIN seq_data s USING (a);"

In [ ]:
# Get only column b
psql -c "${DISABLE_OTHER_JOINS} EXPLAIN SELECT r.b, s.b FROM rand_data r JOIN seq_data s USING (a);"